In [ ]:
import boto3
import sagemaker

In [ ]:
from sagemaker import get_execution_role

sagemaker_role = get_execution_role()
sagemaker_session = sagemaker.Session()
sm_client = sagemaker_session.boto_session.client('sagemaker')


In [ ]:
client = boto3.client('sagemaker-runtime')

endpoint_name = "titanic-driverlessai-mojo-1-6-2"

In [ ]:
ModelName='Driverlessai-MOJO'

#The model can be downloaded from the driverlessAI application. Unzip the 
# file and create a tar.gz. This is to help you package the license file 
# along with the MOJO. This license can also be added as an environment variable. 
# See below.
# Note: The license requirement will be removed and be billed through Sagemaker
ModelLocationInS3 = 's3://yourbucketname/mojo.tar.gz'

response = sm_client.create_model(
    ModelName=ModelName,
    PrimaryContainer={
            'ContainerHostname': 'dai162-rest',
            'Image': '524466471676.dkr.ecr.us-east-1.amazonaws.com/driverlessaidemo:latest',
            'ModelDataUrl': ModelLocationInS3,
            'Environment': {
                'DRIVERLESS_AI_LICENSE_KEY': 'gNpKIUSXh01CcdRhmHMtmXOtn04UJnxp7iNUt70XxqpMdF2fMgVhDPx3gjObzHqo1tv8kcPBqsi-NJQyWs4Vr92FZv-sq7u2HPzX9M2dPPYaBrlWbTxWSrUmnUmDPQClorojsVmsvzT4KIZwGmmyNZADrxaN5b-x4_nfs8WiZnJUxR5BtsC3WC0BEn5IFVeyj7Dru1xI0SltYmVBVlhjH-Jho2kvMzmqaMi77_gjc8NSqdJyaqWFcuWdQB2Xsj3c9hgAty9SVbE1OcEeJrvutE30zHr6NzEvppkHifUIbG3LUtTeMojYeFwbNc2OJDcBQbjdp4F_Ugu5C7yt1Wd1YmxpY2Vuc2VfdmVyc2lvbjoxCnNlcmlhbF9udW1iZXI6MjkKbGljZW5zZWVfb3JnYW5pemF0aW9uOkgyTy5haQpsaWNlbnNlZV9lbWFpbDp0b21rQGgyby5haQpsaWNlbnNlZV91c2VyX2lkOjI5CmlzX2gyb19pbnRlcm5hbF91c2U6dHJ1ZQpjcmVhdGVkX2J5X2VtYWlsOnRvbWtAaDJvLmFpCmNyZWF0aW9uX2RhdGU6MjAxOC8xMS8yNgpwcm9kdWN0OkRyaXZlcmxlc3NBSQpsaWNlbnNlX3R5cGU6ZGV2ZWxvcGVyCmV4cGlyYXRpb25fZGF0ZToyMDE5LzA3LzAxCg=='
            }
    },
    ExecutionRoleArn='arn:aws:iam::524466471676:role/service-role/AmazonSageMaker-ExecutionRole-20180909T081878')
print(response['ModelArn'])

In [ ]:
endpoint_config_name = 'DriverlessAIEndpointConfig'
print(endpoint_config_name)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialInstanceCount':1,
        'ModelName':ModelName,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])


In [ ]:
endpoint_name = 'DriverlessAIEndpoint'
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

try:
    sm_client.get_waiter('endpoint_in_service').wait(EndpointName=endpoint_name)
finally:
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Arn: " + resp['EndpointArn'])
    print("Create endpoint ended with status: " + status)

    if status != 'InService':
        message = sm_client.describe_endpoint(EndpointName=endpoint_name)['FailureReason']
        print('Training failed with the following error: {}'.format(message))
        raise Exception('Endpoint creation did not succeed')

In [ ]:
content_type = "application/octet-stream"

payload = """
{
    "pclass":"2.0",
    "sex":"male",
    "age":"32",
    "sibsp":"4",
    "parch":"2",
    "ticket":"113501",
    "fare":"7.5",
    "cabin":"Q",
    "embarked":"10",
    "boat":"nan",
    "body":"1",
    "home_dest":"AB"
}
"""

response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType=content_type,
    Body=payload
    )

print(response)

response_body = response['Body']
print(response_body.read().decode('utf-8'))
